# Datenanalyse Seattle Library

In [250]:
import urllib.request
import pandas as pd
import matplotlib.pyplot as plt
import csv

## (1) Titel und Autor für meistausgeliehene Medien im Januar 2023

### Daten runterladen

Daten werden schon bei Abfrage bei der API gefiltert nach digitalen und physischen Medien, Entleihjahr und -monat und Mindestanzahl von Ausleihen. Die Mindestanzahl wurde durch Testen so ermittelt, dass die Ausgabe unter den maximal 10.000 Einheiten blieb.
Die Query wurde encoded mithilfe von [https://www.coderstool.com/querystring-encode](https://www.coderstool.com/querystring-encode)

In [254]:
# Query: Ausleihen im Januar 2023 von physischen Medien mit Ausleihzahl von mindestens 6
url = f"https://data.seattle.gov/resource/tmmm-ytt6.csv?$where=usageclass='Physical'&checkoutyear=2023&checkoutmonth=01%20AND%20checkouts%3E6&$limit=10000"
local_file_phys = "physical_2023-01.csv"
urllib.request.urlretrieve(url, local_file_phys)

('physical_2023-01.csv', <http.client.HTTPMessage at 0x7983fbb5dd00>)

In [255]:
# Query: Ausleihen im Januar 2023 von digitalen Medien mit Ausleihzahl von mindestens 8
url = "https://data.seattle.gov/resource/tmmm-ytt6.csv?$where=usageclass='Digital'&checkoutyear=2023&checkoutmonth=01%20AND%20checkouts%3E8&$limit=10000"
local_file_dig = "digital_2023-01.csv"
urllib.request.urlretrieve(url, local_file_dig)

('digital_2023-01.csv', <http.client.HTTPMessage at 0x7983fbb5d400>)

In [256]:
# Laden in Dateframes
df_physical = pd.read_csv(local_file_phys)
df_digital = pd.read_csv(local_file_dig)

In [257]:
df_physical.shape

(9975, 12)

In [258]:
df_digital.shape

(8867, 12)

### Die beiden meistausgeliehenen Objekte (physisch und digital)

In [260]:
max_phys_int = df_physical["checkouts"].max()
max_dig_int = df_digital["checkouts"].max()

In [261]:
print("Maximale Ausleihzahl eines physischen Objekts: " + str(max_phys_int))
print("Maximale Ausleihzahl eines digitalen Objekts: " + str(max_dig_int))

Maximale Ausleihzahl eines physischen Objekts: 2115
Maximale Ausleihzahl eines digitalen Objekts: 791


In [262]:
max_phys_row = df_physical.loc[df_physical["checkouts"] == max_phys_int]
max_dig_row = df_digital.loc[df_digital["checkouts"] == max_dig_int]

In [263]:
max_phys_row

,usageclass,checkouttype,materialtype,checkoutyear,checkoutmonth,checkouts,title,isbn,creator,subjects,publisher,publicationyear
1123,Physical,Horizon,MIXED,2023,1,2115,Headphones / Seattle Public Library.,NaN,NaN,NaN,Seattle Public Library,NaN


In [264]:
max_dig_row

,usageclass,checkouttype,materialtype,checkoutyear,checkoutmonth,checkouts,title,isbn,creator,subjects,publisher,publicationyear
5137,Digital,OverDrive,AUDIOBOOK,2023,1,791,101 Essays That Will Change the Way You Think ...,9.781977e+12,Brianna Wiest,"Essays, Philosophy, Self-Improvement, Nonfiction","Tantor Media, Inc",2021


Kombination beider Zeilen in eine Tabelle (hat keinen tieferen Sinn, ist nur zum concat-Ausprobieren)

In [265]:
type(max_dig_row)

pandas.core.frame.DataFrame

In [266]:
max_df_combined = pd.concat([max_phys_row, max_dig_row])
max_df_combined

/tmp/ipykernel_47231/331941362.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  max_df_combined = pd.concat([max_phys_row, max_dig_row])


,usageclass,checkouttype,materialtype,checkoutyear,checkoutmonth,checkouts,title,isbn,creator,subjects,publisher,publicationyear
1123,Physical,Horizon,MIXED,2023,1,2115,Headphones / Seattle Public Library.,NaN,NaN,NaN,Seattle Public Library,NaN
5137,Digital,OverDrive,AUDIOBOOK,2023,1,791,101 Essays That Will Change the Way You Think ...,9.781977e+12,Brianna Wiest,"Essays, Philosophy, Self-Improvement, Nonfiction","Tantor Media, Inc",2021


### Die zehn meistausgeliehenen physischen Objekte

In [267]:
ranking_checkouts_phys_series = df_physical["checkouts"].nlargest(10)

In [269]:
type(ranking_checkouts_phys_series)

pandas.core.series.Series

In [270]:
print(ranking_checkouts_phys_series)

1123    2115
4770     841
8074     485
2481     432
5942     426
4870     416
9268     416
9699     381
301      377
1176     373
Name: checkouts, dtype: int64


In [271]:
df_subset_most_checkouts_phys = df_physical.nlargest(10,"checkouts")
df_subset_most_checkouts_dig = df_digital.nlargest(10,"checkouts")

In [272]:
type(df_subset_most_checkouts_dig)

pandas.core.frame.DataFrame

### Die zehn meistgeliehenen physischen Objekte im Januar 2023 mit Art des Objekts, Titel, Urheber und Anzahl der Ausleihen

In [127]:
df_subset_most_checkouts_phys[["materialtype", "title", "creator", "checkouts"]]

,materialtype,title,creator,checkouts
1123,MIXED,Headphones / Seattle Public Library.,NaN,2115
4770,MIXED,SPL HotSpot : connecting Seattle / [distribute...,NaN,841
8074,BOOK,Spare / Prince Harry.,"Harry, Prince, Duke of Sussex, 1984-",485
2481,BOOK,Lessons in chemistry / Bonnie Garmus.,"Garmus, Bonnie",432
5942,BOOK,Our missing hearts : a novel / Celeste Ng.,"Ng, Celeste",426
4870,BOOK,I'm glad my mom died / Jennette McCurdy.,"McCurdy, Jennette, 1992-",416
9268,BOOK,The light we carry : overcoming in uncertain t...,"Obama, Michelle, 1964-",416
9699,BOOK,Dinner in one : exceptional & easy one-pan mea...,"Clark, Melissa",381
301,BOOK,Foster / Claire Keegan.,"Keegan, Claire",377
1176,BOOK,The passenger / Cormac McCarthy.,"McCarthy, Cormac, 1933-",373


### Die zehn meistgeliehenen digitalen Objekte im Januar 2023 mit Art des Objekts, Titel, Urheber und Anzahl der Ausleihen

In [128]:
df_subset_most_checkouts_dig[["materialtype", "title", "creator", "checkouts"]]

,materialtype,title,creator,checkouts
5137,AUDIOBOOK,101 Essays That Will Change the Way You Think ...,Brianna Wiest,791
441,AUDIOBOOK,"Braiding Sweetgrass: Indigenous Wisdom, Scient...",Robin Wall Kimmerer,729
1426,EBOOK,Lessons in Chemistry: A Novel,Bonnie Garmus,681
660,EBOOK,"Tomorrow, and Tomorrow, and Tomorrow: A novel",Gabrielle Zevin,667
5570,AUDIOBOOK,I'm Glad My Mom Died (unabridged),Jennette McCurdy,564
3172,AUDIOBOOK,Spare (unabridged),"Prince Harry, The Duke of Sussex",533
8709,AUDIOBOOK,A Court of Thorns and Roses: A Court of Thorns...,Sarah J. Maas,515
4457,EBOOK,Cloud Cuckoo Land: A Novel,Anthony Doerr,478
8171,AUDIOBOOK,"Tomorrow, and Tomorrow, and Tomorrow: A novel ...",Gabrielle Zevin,466
6541,EBOOK,The Last Thing He Told Me: A Novel,Laura Dave,440


## (2) Wie viele Comics mit Erscheinungsjahr 2005 finden sich in der Collection? 

### ILS Data Dictionary laden

In [131]:
url = "https://data.seattle.gov/resource/pbt3-ytbc.json"
local_file = "seattle_dd.json"
urllib.request.urlretrieve(url, local_file)

('seattle_dd.json', <http.client.HTTPMessage at 0x798400529ca0>)

In [136]:
df_data_dict = pd.read_json("seattle_dd.json")
df_data_dict

,code,description,code_type,format_group,format_subgroup,category_group,age_group,category_subgroup
0,cazover,CA7-zine collection oversize,ItemCollection,Print,Book,Periodical,Adult,NaN
1,caziner,CA7-zine collection reference,ItemCollection,Print,Book,Periodical,Adult,NaN
2,cazval,CA7-zine collection valuable mat.,ItemCollection,Print,Book,Periodical,Adult,NaN
3,napass,NA-Discovery Pass,ItemCollection,Media,Pass,Miscellaneous,Adult,NaN
4,ccbocd,CC - Children's Books on CD,ItemCollection,Media,Audiobook Disc,Fiction,Juvenile,NaN
...,...,...,...,...,...,...,...,...
620,acblu,Blu-ray: Adult/YA,ItemType,Media,Video Disc,NaN,Adult,NaN
621,ncblu,NC-Children's Blu-rays,ItemCollection,Media,Video Disc,Fiction,Juvenile,NaN
622,cazinec,ZAPP zine collection circulating,ItemCollection,Print,Book,Miscellaneous,Adult,NaN
623,cavalf,CA0-Valuable Material Folio,ItemCollection,NaN,NaN,NaN,NaN,NaN


### Tabelle für Codes von Interlibrary Loans

In [155]:
url = "https://data.seattle.gov/resource/pbt3-ytbc.json?category_group='Interlibrary%20Loan'"
local_file = "seattle_il.json"
urllib.request.urlretrieve(url, local_file)

('seattle_il.json', <http.client.HTTPMessage at 0x7983fe948350>)

In [156]:
df_data_dict["category_group"].unique()

array(['Periodical', 'Miscellaneous', 'Fiction', 'Nonfiction',
       'Reference', nan, 'Interlibrary Loan', 'Temporary', 'Analytic',
       'WTBBL', 'Uncataloged', 'Language', 'On Order'], dtype=object)

In [164]:
df_il = pd.read_json("seattle_il.json")
df_il

,code,description,code_type,category_group,format_group,age_group
0,ill,Interlibrary Loan,Location,Interlibrary Loan,NaN,NaN
1,swill,Interlibrary Loan,ItemCollection,Interlibrary Loan,Other,Adult
2,dcillb,Ill Borrowed,ItemType,Interlibrary Loan,Other,Adult
3,dcilll,Ill Lending,ItemType,Interlibrary Loan,Other,Adult


### Tabelle für Codes der Subgroups

In [138]:
df_data_dict["category_subgroup"].unique()

array([nan, 'Biography', 'Peak Picks', 'Comic', 'Large Print', 'Holiday',
       'Picture', 'ESL'], dtype=object)

In [160]:
url = "https://data.seattle.gov/resource/pbt3-ytbc.json?$where=category_subgroup%20IS%20NOT%20NULL"
local_file = "seattle_sub.json"
urllib.request.urlretrieve(url, local_file)

('seattle_sub.json', <http.client.HTTPMessage at 0x7983fe84b2c0>)

In [235]:
df_sub = pd.read_json("seattle_sub.json")
df_comics = df_sub[df_sub["category_subgroup"] == "Comic"]

In [236]:
comic_codes = df_comics["code"].unique()

In [237]:
comic_codes

array(['cccomic', 'nacomic', 'cacomic', 'nccomic', 'cycomic', 'nycomic'],
      dtype=object)

In [242]:
for x in comic_codes:
    print(x)

cccomic
nacomic
cacomic
nccomic
cycomic
nycomic


In [238]:
type(comic_codes)

numpy.ndarray

### Load Library Collection Inventory

In [304]:
url = "https://data.seattle.gov/resource/6vkj-f5xf.json?$limit=10000"
local_file = "seattle-lci.json"
urllib.request.urlretrieve(url, local_file)
#seattle_json_raw = urllib.request.urlopen(url).read()

('seattle-lci.json', <http.client.HTTPMessage at 0x7983fb14db20>)

In [305]:
df_seattle_lci = pd.read_json(local_file)

In [306]:
lci_table_reduced = df_seattle_lci[["bibnum", "title", "author", "itemcollection"]]

In [307]:
lci_table_reduced

,bibnum,title,author,itemcollection
0,2570004,Climate change : picturing the science / Gavin...,"Schmidt, Gavin",nanf
1,2641788,The big book of Civil War sites : from Fort Su...,NaN,nanf
2,3117453,The three little pigs fight back / by Annie Au...,"Auerbach, Annie",ncpic
3,3002472,Some velvet morning / Tribeca Film ; Cristile ...,NaN,nadvd
4,2830564,Shrines / Purity Ring.,Purity Ring (Musical group),nacd
...,...,...,...,...
9995,220001,"Pills, pesticides & profits : the internationa...",NaN,cs6
9996,3117379,Understanding chronic fatigue syndrome : an in...,"Ali, Naheed, 1981-",canf
9997,2936698,Missing sisters / Gregory Maguire.,"Maguire, Gregory",ncfic
9998,3082942,"Yes, we can print that! : 25 years at Day Moon...",NaN,casea


In [309]:
lci_tables_combined = pd.merge(lci_table_reduced, df_data_dict, how="left", on="temcollection)

NameError: name 'itemcollection' is not defined

## ISBN

### Daten runterladen

In [19]:
url = "https://data.seattle.gov/resource/tmmm-ytt6.csv?$where=(starts_with(isbn,%20'978')%20OR%20starts_with(isbn,'979'))%20AND%20checkoutyear=2023&$limit=10000"
local_file = "checkout.csv"
urllib.request.urlretrieve(url, local_file)

('checkout.csv', <http.client.HTTPMessage at 0x798406dbc6e0>)

In [20]:
df_seattle_isbn = pd.read_csv(local_file)

In [21]:
df_seattle_isbn

,usageclass,checkouttype,materialtype,checkoutyear,checkoutmonth,checkouts,title,isbn,creator,subjects,publisher,publicationyear
0,Digital,OverDrive,EBOOK,2023,3,10,Self-Portrait with Nothing,9781250820853,Aimee Pokwatka,"Fiction, Literature, Mystery, Science Fiction",Macmillan Publishers,2022
1,Digital,OverDrive,EBOOK,2023,3,6,The Fervor,9780593328347,Alma Katsu,"Fiction, Horror, Thriller, Historical Fiction","Penguin Group (USA), Inc.",2022
2,Digital,OverDrive,EBOOK,2023,3,1,Heidi Heckelbeck and the Christmas Surprise,9781442481268,Wanda Coven,Juvenile Fiction,"Simon & Schuster, Inc.",2014
3,Digital,OverDrive,EBOOK,2023,3,2,Selected Short Fiction,9780786512362,Charles Dickens,"Classic Literature, Fiction, Short Stories","Penguin Group (USA), Inc.",2011
4,Digital,OverDrive,EBOOK,2023,3,3,Bellweather Rhapsody,9780544133471,Kate Racculia,"Fiction, Literature, Mystery",HarperCollins Publishers Inc.,2014
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,Digital,OverDrive,AUDIOBOOK,2023,1,2,The Teacher's Funeral: A Comedy in Three Parts...,9780739345238,Richard Peck,"Historical Fiction, Young Adult Fiction, Young...",Books on Tape,2006
9996,Digital,OverDrive,EBOOK,2023,1,1,We All Fall Down: The River City Duology Serie...,9780374314330,Rose Szabo,"Fantasy, Young Adult Fiction, LGBTQIA+ (Fiction)",Macmillan Publishers,2022
9997,Digital,OverDrive,EBOOK,2023,1,1,The Last Days of Socrates,9780141965888,Plato,"Classic Literature, Fiction",Penguin Random House UK,2010
9998,Digital,OverDrive,EBOOK,2023,1,1,The Children of Henry VIII,9780307806864,Alison Weir,"Biography & Autobiography, History, Nonfiction","Random House, Inc.",2013


In [22]:
df_seattle_isbn.keys()

Index(['usageclass', 'checkouttype', 'materialtype', 'checkoutyear',
       'checkoutmonth', 'checkouts', 'title', 'isbn', 'creator', 'subjects',
       'publisher', 'publicationyear'],
      dtype='object')

In [23]:
df_seattle_isbn["materialtype"].unique()

array(['EBOOK', 'AUDIOBOOK', 'BOOK', 'VIDEODISC', 'SOUNDDISC'],
      dtype=object)

In [24]:
df_seattle_isbn[df_seattle_isbn["materialtype"]=="AUDIOBOOK"]

,usageclass,checkouttype,materialtype,checkoutyear,checkoutmonth,checkouts,title,isbn,creator,subjects,publisher,publicationyear
6,Digital,OverDrive,AUDIOBOOK,2023,3,2,Nine Inches: Stories (unabridged),9781427235305,Tom Perrotta,"Fiction, Literature, Short Stories",Macmillan Audio,2013
7,Digital,OverDrive,AUDIOBOOK,2023,3,1,Undiscovered Country (unabridged),9781415956243,Lin Enger,"Fiction, Literature",Books on Tape,2018
12,Digital,OverDrive,AUDIOBOOK,2023,3,1,Just One Touch (unabridged),9780062674326,Maya Banks,"Fiction, Romance, Suspense",HarperCollins Publishers Inc.,2017
17,Digital,OverDrive,AUDIOBOOK,2023,3,2,First Born: A Novel (unabridged),9781797142814,Will Dean,"Fiction, Suspense, Thriller",Simon & Schuster - Audiobooks,2022
18,Digital,OverDrive,AUDIOBOOK,2023,3,5,Berlin Alexanderplatz (unabridged),9781977353825,Alfred Doblin,"Fiction, Literature","Tantor Media, Inc",2020
...,...,...,...,...,...,...,...,...,...,...,...,...
9987,Digital,OverDrive,AUDIOBOOK,2023,1,2,The Wires of War: Technology and the Global St...,9781797132723,Jacob Helberg,"Computer Technology, Politics, Nonfiction",Simon & Schuster - Audiobooks,2021
9988,Digital,OverDrive,AUDIOBOOK,2023,1,9,Winter's Orbit (unabridged),9781250790194,Everina Maxwell,"Fiction, Science Fiction",Macmillan Audio,2021
9990,Digital,OverDrive,AUDIOBOOK,2023,1,8,"They Do It With Mirrors: Miss Marple Series, B...",9781408484890,Agatha Christie,"Fiction, Mystery, Thriller",Penguin Random House UK,2021
9992,Digital,OverDrive,AUDIOBOOK,2023,1,45,Portrait of a Thief: A Novel (unabridged),9780593510964,Grace D. Li,"Fiction, Literature, Mystery",Books on Tape,2022


In [25]:
df_seattle_isbn[df_seattle_isbn["materialtype"]=="VIDEODISC"]

,usageclass,checkouttype,materialtype,checkoutyear,checkoutmonth,checkouts,title,isbn,creator,subjects,publisher,publicationyear
4391,Physical,Horizon,VIDEODISC,2023,1,2,Finding your roots. Season 2 / with Henry Loui...,9781627890946,NaN,"Genealogy, Celebrities Genealogy, Celebrities ...","Distributed by PBS Distribution,",[2014]
4496,Physical,Horizon,VIDEODISC,2023,1,1,Craft in America. Celebration / executive prod...,9781627894258,NaN,"Handicraft United States History 20th century,...","Distributed by PBS Distribution,",c2016.
7102,Physical,Horizon,VIDEODISC,2023,1,5,The fugitive kind / 20th Century Fox ; Metro-G...,9781604652703,NaN,"Triangles Interpersonal relations Drama, Drift...","Criterion Collection,",2010.
8999,Physical,Horizon,VIDEODISC,2023,1,2,Defiant Requiem / a Partisan Pictures producti...,9781608839179,NaN,"Sch„chter Raphael, Verdi Giuseppe 1813 1901 Me...","PBS Distribution,",[2013]


In [26]:
df_seattle_isbn["usageclass"].value_counts()

usageclass
Digital     9942
Physical      58
Name: count, dtype: int64

### Download physical with 978 ISBN

In [27]:
url = "https://data.seattle.gov/resource/tmmm-ytt6.csv?$where=(starts_with(isbn,%20'978')%20OR%20starts_with(isbn,'979'))%20AND%20usageclass='Physical'&$limit=10000"
local_file = "checkout2.csv"
urllib.request.urlretrieve(url, local_file)

('checkout2.csv', <http.client.HTTPMessage at 0x798406be2180>)

In [28]:
df_physical = pd.read_csv(local_file)

In [29]:
df_physical

,usageclass,checkouttype,materialtype,checkoutyear,checkoutmonth,checkouts,title,isbn,creator,subjects,publisher,publicationyear
0,Physical,Horizon,BOOK,2022,5,17,Macrina Bakery : seasons / written by Leslie M...,"9780578614939, 9789910387715, 9910387719","Mackie, Leslie","Seasonal cooking, Macrina Bakery and Cafe, Foo...","Macrina Bakery,",[2019]
1,Physical,Horizon,BOOK,2022,5,1,The red apple / Feridun Oral.,"9789888240005, 9888240005","Oral, Feridun","Friendship Juvenile fiction, Cooperation Juven...","Minedition (Michael Neugebauer Publishing Ltd.),",[2015]
2,Physical,Horizon,BOOK,2022,5,1,Tai cai ben wei : Huang Wanling de cai wei tan...,"9789869899659, 986989965X","Huang, Wanling","Cooking Chinese Taiwan style, Snack foods Taiw...","Xie le wen hua you xian gong si,",2021.
3,Physical,Horizon,BOOK,2022,5,1,Mr. Hollyberry's Christmas gift / by Kate West...,"9789888341627, 9888341626","Westerlund, Kate","Snowmen Juvenile fiction, Cats Juvenile fictio...","Michael Neugebauer Publishing,",2018.
4,Physical,Horizon,BOOK,2022,5,3,Geek love / Katherine Dunn.,9780375713347,"Dunn, Katherine, 1945-2016","Carnival owners Fiction, Circus performers Fic...","Vintage Books,",2002.
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,Physical,Horizon,BOOK,2023,4,1,Adaku & other stories / Ken Saro-Wiwa.,9782460117,"Saro-Wiwa, Ken, 1941-1995",NaN,"Saros International Publishers,",1989.
9996,Physical,Horizon,BOOK,2023,4,5,The forgotten crayon / Yoko Maruyama.,"9789888341986, 9888341987","Maruyama, Yoko (Children's author)","Crayons Juvenile fiction, Picture books","Michael Neugebauer Publishing,",2019.
9997,Physical,Horizon,BOOK,2023,4,3,Korean children's favorite stories / retold by...,9780804835916,"Kim, So-un, 1907-1981","Tales Korea Juvenile literature, Folklore Kore...","Tuttle Pub.,",[2004]
9998,Physical,Horizon,VIDEODISC,2023,4,31,Bergman island / IFC Films and CG Cinéma prese...,"9789798886072, 9798886070, 9798886070125",NaN,"Bergman Ingmar 1918 2007 Drama, Independent fi...","Criterion Collection,",[2023]
